# Программирование на Python

*Алла Тамбовцева, НИУ ВШЭ*

## Работа с API ВКонтакте: собираем посты со стены, комментарии к постам и информацию пользователях

## Подготовка к работе

Загружаем модули и библиотеки, необходимые для работы:

In [1]:
import requests
import time
import pandas as pd

Для начала давайте посмотрим на документацию API и посмотрим, как к нему формировать запросы: https://dev.vk.com/api/api-requests.
По [инструкции](https://allatambov.github.io/pypolit/vk-auth.pdf) мы можем получить доступ к API, создадим приложение и скопируем его ID, чтобы получить ссылку для токена доступа:

In [ ]:
# если нет Управление в меню во ВКонтакте
# https://vk.com/apps?act=manage

In [2]:
app_id = input("Enter your client id: ")
url = f"https://oauth.vk.com/authorize?client_id={app_id}&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token"
print(url)

Enter your client id: 51660535
https://oauth.vk.com/authorize?client_id=51660535&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token


Переходим по ссылке выше и копируем ссылку (полностью!) из адресной строки:

In [3]:
full_link = input()

https://oauth.vk.com/blank.html#access_token=vk1.a.VxWyW8j8CWtqnINPP_4WFE6ajy-R3KDRLKW7lGCT0zGUecCz_JhO8ZRpIeP-lLZYdGMU8V70ugHh4CSaAmTlc5Pb53uH9YGaGMBqog9bPXDlD-O3CdKEs_J8zqTotSMmQHMOsr3DOMwkKY7L6TVG8y6Zg60QxDtB0tWkDnWGu7K27yzcxEQ4o5DUZXwTqE6E&expires_in=86400&user_id=20473269


Разбиваем полученную ссылку (сначала по `access_token=`, потом часть после `=` по `&`) и забираем токен в чистом виде:

In [4]:
token = full_link.split("access_token=")[1].split("&")[0]

Ура! Теперь токен доступа у нас есть, всё готово к работе!

## Часть 1: выгружаем посты со стены сообщества

На этом практическом занятии мы будем выгружать посты из сообщества [скалодрома Rock Zona](https://vk.com/rzclimbing). Сохраним в переменные версию API, ссылку для метода работы со стеной сообщества и название сообщества:

In [6]:
v = "5.131"
main_wall = "https://api.vk.com/method/wall.get"
domain = "rzclimbing"

Функция `get()` из библиотеки `requests` умеет подставлять в запрос необходимые параметры и объединять их с помощью `?` и `&`. Сохраним необходимые параметры в виде словаря:

In [7]:
params_wall = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v}

А теперь сформируем запрос и выгрузим результаты в формате JSON – в Python данные в таком формате будут представлены в виде словаря:

In [8]:
req_wall = requests.get(main_wall, params = params_wall)

In [9]:
req_wall.url

'https://api.vk.com/method/wall.get?access_token=vk1.a.VxWyW8j8CWtqnINPP_4WFE6ajy-R3KDRLKW7lGCT0zGUecCz_JhO8ZRpIeP-lLZYdGMU8V70ugHh4CSaAmTlc5Pb53uH9YGaGMBqog9bPXDlD-O3CdKEs_J8zqTotSMmQHMOsr3DOMwkKY7L6TVG8y6Zg60QxDtB0tWkDnWGu7K27yzcxEQ4o5DUZXwTqE6E&domain=rzclimbing&count=100&v=5.131'

In [11]:
json_wall = req_wall.json()
#json_wall

Извлечём из этого большого словаря элемент, который отвечает за общее число постов на стене:

In [12]:
nposts = json_wall['response']['count']
print(nposts)

1554


Теперь извлечём элемент, который хранит результаты – список из маленьких словарей с информацией о постах (1 словарь = 1 пост):

In [13]:
items_wall = json_wall['response']['items']

Посмотрим на один элемент такого списка:

In [17]:
i = items_wall[2]
i

{'donut': {'is_donut': False},
 'comments': {'can_post': 1, 'count': 0, 'groups_can_post': True},
 'marked_as_ads': 0,
 'short_text_rate': 0.8,
 'hash': 'okWpDo78zzuXtmHxzyydcT6796w',
 'type': 'post',
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1686318631,
    'id': 457244264,
    'owner_id': -38936316,
    'access_key': '4379369f1eb513e5f4',
    'post_id': 4072,
    'sizes': [{'height': 75,
      'type': 's',
      'width': 75,
      'url': 'https://sun1-15.userapi.com/impg/84ogn5IDQUeJ2JJu7ER_Qjok3EWxUu-A_u31NQ/WKOmoYEuAKY.jpg?size=75x75&quality=95&sign=09e33bbbe73a1852faddb27c80984ac4&c_uniq_tag=--RJjmNzfnTjnSSrpX09UtZvxeToNDLKQuSxqp7EY9k&type=album'},
     {'height': 130,
      'type': 'm',
      'width': 130,
      'url': 'https://sun1-15.userapi.com/impg/84ogn5IDQUeJ2JJu7ER_Qjok3EWxUu-A_u31NQ/WKOmoYEuAKY.jpg?size=130x130&quality=95&sign=8fc44fa74edb38a4d291510735704ff2&c_uniq_tag=TDFPA9vlW24O1k-gKUkPfr1UMzkFO0r7EKbK7gX7eVk&type=album'},
     {'heig

### Задача 1

Извлеките из элемента `i` следующие компоненты:

* id поста;
* дата поста;
* текст поста;
* число лайков;
* число репостов;
* число просмотров;
* число комментариев.

In [18]:
### YOUR CODE HERE ###
print(i["id"], i["date"], i["text"])
print(i["likes"]["count"], i["reposts"]["count"], 
     i["views"]["count"], i["comments"]["count"])

4072 1686318631 ❓ Как подготовиться к выезду на скалы

🔹  Перед тем, как ехать на скалы, начните посещать скалодром более регулярно. Уделяйте как можно больше внимания технике постановки ног, особенно на маленькие зацепки.  

🔹 Изучите гайдбук района, в который вы поедете. Составьте примерный план, выберите трассы для проектов. Ознакомьтесь с подходами к трассам, правилами и ограничениями района и составьте список необходимого снаряжения.  

🔹 Научитесь работать с веревкой или освежите свои знания. Возьмите пару занятий с профессиональным инструктором, чтобы убедиться, что вы умеете правильно вязать узлы, страховать, провешивать и снимать трассы. 

🔸 Очень внимательно выбирайте напарника. Это должен быть надежный человек, с которым вам приятно общаться и в квалификации которого вы уверены.  

🔸Убедитесь в том, что вы имеете все необходимое снаряжение, длина веревки соответствует длине трасс, у вас достаточно оттяжек. Убедитесь, что ваше снаряжение в хорошем состоянии и у него не вышел 

### Задача 2

Изучить один пост и понять, что нам от него нужно, это хорошо, но, конечно, мы захотим выгрузить все посты сразу, а уже потом разобраться, какую информацию о них нам оставить. Ограничения данного API таковы, что за один раз мы можем выгрузить только 100 постов. Хорошие новости: каждый раз при выгрузке мы можем начинать с того поста, на котором закончили, то есть сначала выгрузить первые 100 постов, потом – следующие 100 постов, и так до тех пор, пока не заполучим все. 

Общее число постов сохранено в `nposts`. Посчитайте, сколько раз нужно будет выполнить выгрузку по 100 постов, чтобы собрать все тексты, и сохраните его в переменную `iterate`.

In [19]:
iterate = nposts // 100 + 1  # целочисленное деление + 1 

### Задача 3

Прочитайте в документации к API ВКонтакте про аргумент `offset` в методе `wall.get`. Используя полученную информацию и блоки кода ниже, выгрузите и сохраните в список `items_all` данные по всем постам на стене сообщества.

**Подсказка:** чтобы расширять список правильным образом, используйте метод `.extend()`, а не `.append()`, он добавляет не один элемент, а сразу несколько.

In [20]:
params_wall_long = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v,
              "offset" : 0}

In [21]:
items_all = []

for i in range(iterate):
    req_wall_long = requests.get(main_wall, params = params_wall_long)
    json_wall_long = req_wall_long.json()
    items_wall_long = json_wall_long['response']['items']
    
    # обновляем список items_all
    # обновляем парметр offset - увеличиваем на 100
    items_all.extend(items_wall_long)
    params_wall_long["offset"] = params_wall_long["offset"] + 100
    
    
    time.sleep(1.5)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


Проверяем длину списка – все ли посты собраны:

In [22]:
len(items_all)

1554

Не отбираем ничего на этом этапе, просто превращаем список словарей в датафрейм:

In [23]:
df = pd.DataFrame(items_all)
df.head(10)

,donut,comments,marked_as_ads,short_text_rate,hash,type,attachments,date,from_id,id,...,post_type,reposts,text,views,carousel_offset,edited,copy_history,geo,signer_id,zoom_text
0,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,DFtdCebqVmBxDimFGnDqgVrCF4Y,post,"[{'type': 'video', 'video': {'response_type': ...",1686324040,-38936316,4076,...,post,"{'count': 0, 'user_reposted': 0}",,{'count': 312},NaN,NaN,NaN,NaN,NaN,NaN
1,{'is_donut': False},"{'can_post': 1, 'count': 1, 'groups_can_post':...",0,0.8,E0Z_ygeZYfGEssLdBddXmBbEjU4,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1686319330,-38936316,4073,...,post,"{'count': 0, 'user_reposted': 0}",Выберите правдивые утверждения о нашем тренере...,{'count': 196},0.0,1.686319e+09,NaN,NaN,NaN,NaN
2,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,okWpDo78zzuXtmHxzyydcT6796w,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1686318631,-38936316,4072,...,post,"{'count': 0, 'user_reposted': 0}",❓ Как подготовиться к выезду на скалы\n\n🔹 Пе...,{'count': 194},NaN,1.686319e+09,NaN,NaN,NaN,NaN
3,{'is_donut': False},"{'can_post': 1, 'count': 2, 'groups_can_post':...",0,0.8,flx3QwYVJJ9dz9pObZHEkp8EuHw,post,"[{'type': 'video', 'video': {'response_type': ...",1686220008,-38936316,4071,...,post,"{'count': 0, 'user_reposted': 0}",,{'count': 736},NaN,NaN,NaN,NaN,NaN,NaN
4,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,UqUhWcx1m0asW5B4Jvsqzljc_lE,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1686161707,-38936316,4068,...,post,"{'count': 6, 'user_reposted': 0}",Сегодня мы познакомимся с достаточно эффективн...,{'count': 614},NaN,NaN,NaN,NaN,NaN,NaN
5,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,ijlmziVBz7nBKkEAcW3R48clBrg,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1686071217,-38936316,4067,...,post,"{'count': 1, 'user_reposted': 0}",Хорошему скалодрому - хорошие трассы! ☺️\n\n🔥 ...,{'count': 533},0.0,NaN,NaN,NaN,NaN,NaN
6,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,pK6YMrqutvwjGX21f-QkZ2_TSHg,post,"[{'type': 'video', 'video': {'response_type': ...",1685975486,-38936316,4064,...,post,"{'count': 1, 'user_reposted': 0}",,{'count': 717},NaN,NaN,NaN,NaN,NaN,NaN
7,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,ho9aypUx2i9IzVpphQq44KqH420,post,"[{'type': 'video', 'video': {'response_type': ...",1685902161,-38936316,4063,...,post,"{'count': 0, 'user_reposted': 0}",,{'count': 740},NaN,NaN,NaN,NaN,NaN,NaN
8,{'is_donut': False},"{'can_post': 1, 'count': 4, 'groups_can_post':...",0,0.8,mWLIHJbUzrR_Lmf5FxjOTVlO0TQ,post,"[{'type': 'photo', 'photo': {'album_id': -7, '...",1685624479,-38936316,4058,...,post,"{'count': 11, 'user_reposted': 0}",Как возраст влияет на скалолазание.\n\n🔹Статис...,{'count': 2051},NaN,NaN,NaN,NaN,NaN,NaN
9,{'is_donut': False},"{'can_post': 1, 'count': 0, 'groups_can_post':...",0,0.8,o9o6VAJ2X5lHwGXmDs01SnQzgEA,post,"[{'type': 'video', 'video': {'response_type': ...",1685552013,-38936316,4055,...,post,"{'count': 0, 'user_reposted': 0}",,{'count': 636},NaN,NaN,NaN,NaN,NaN,NaN


### Задача 4

Создайте на основе датафрейма `df` новый датафрейм `small` со следующими столбцами:

* id поста (`id`);
* дата поста (`date`);
* текст поста (`text`);
* число лайков (`nlikes`);
* число просмотров (`nviews`);
* число комментариев (`ncomments`).

**Подсказка:** отберите сначала все столбцы с нужной информацией, напишите функцию для извлечения только числа лайков/просмотров/комментариев и примените её, удалите лишние столбцы.

In [24]:
# выбираем нужные столбцы
small = df[["id", "date", "text", "likes", "reposts", "comments"]]

In [25]:
# нужно научиться работать с ячейками такого вида
small["likes"][0]

{'can_like': 1,
 'count': 9,
 'user_likes': 0,
 'can_publish': 1,
 'repost_disabled': False}

In [26]:
# пишем lambda-функцию, которая принимает на вход ячейку (x)
# и забирает оттуда значение по ключу count

small["nlikes"] = small["likes"].apply(lambda x: x["count"])
small["nreposts"] = small["reposts"].apply(lambda x: x["count"])
small["ncomments"] = small["comments"].apply(lambda x: x["count"])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [27]:
# удаляем лишние столбцы
# те, откуда извлекли count, больше не нужны
# inplace = True: сохраняем изменения сразу в small

small.drop(columns = ["reposts", "comments", "likes"], inplace =True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [28]:
small.head()

,id,date,text,nlikes,nreposts,ncomments
0,4076,1686324040,,9,0,0
1,4073,1686319330,Выберите правдивые утверждения о нашем тренере...,4,0,1
2,4072,1686318631,❓ Как подготовиться к выезду на скалы\n\n🔹 Пе...,8,0,0
3,4071,1686220008,,25,0,2
4,4068,1686161707,Сегодня мы познакомимся с достаточно эффективн...,13,6,0


### Задача 5

Выберите только те строки в полученном датафрейме, которые соответствуют постам с числом комментариев больше 0, и сохраните их в датафрейм `with_comm`.

In [29]:
with_comm = small[small["ncomments"] > 0] 
with_comm.shape

(515, 6)

## Часть 2: собираем комментарии к постам

В целях экономии времени не будем брать весь датафрейм `with_comm`, для примера отберем первые 10 строк:

In [30]:
with_comm10 = with_comm.head(10) 

Заберем id постов и преобразуем их в список:

In [31]:
ids = list(with_comm10["id"])

Возьмем один id для примера и сформулируем запрос для получения комментариев к посту с этим id:

In [32]:
i = 3782

# метод wall.getComments
# owner_id: id сообщества
# post_id: id поста
# v: версия API, все та же
# access_token: токен доступа, все тот же
# count: число комментариев, берем 100, max возможное за раз
# thread_items_count: число ответов на комментарий, 10, max за раз

main_comm = "https://api.vk.com/method/wall.getComments"
params_comm = {"owner_id" : -38936316,
              "post_id" : i,
              "v" : v,
              "access_token" : token,
              "count" : 100, 
              "thread_items_count" : 10}

Формулируем запрос, отправляем и смотрим на результат в виде словаря:

In [33]:
req = requests.get(main_comm, params = params_comm)
req.json() 

{'response': {'count': 4,
  'items': [{'id': 3783,
    'from_id': 527923327,
    'date': 1669311085,
    'text': 'Мотивация не нужна, нужна дисциплина.',
    'post_id': 3782,
    'owner_id': -38936316,
    'parents_stack': [],
    'thread': {'count': 1,
     'items': [{'id': 3784,
       'from_id': 2311030,
       'date': 1669314785,
       'text': '[id527923327|Николай], 👍',
       'post_id': 3782,
       'owner_id': -38936316,
       'parents_stack': [3783],
       'reply_to_user': 527923327,
       'reply_to_comment': 3783}],
     'can_post': True,
     'show_reply_button': True,
     'groups_can_post': True}},
   {'id': 3785,
    'from_id': 2847724,
    'date': 1669373425,
    'text': 'Мотивация тоже нужна)',
    'post_id': 3782,
    'owner_id': -38936316,
    'parents_stack': [],
    'thread': {'count': 1,
     'items': [{'id': 3788,
       'from_id': 527923327,
       'date': 1669400413,
       'text': '[id2847724|Татьяна], мотивация лишь временный фактор, дисциплина - вечна. Дис

### Задача 6

Давайте считать, что 100 самых новых комментариев и 10 ответов на них нам достаточно (если нет – аргумент `offset` есть, задача аналогична предыдущей по сбору постов). Повторите эту операцию выше для всех id в списке `ids` и сформируйте список с результатами. Преобразуйте результат в датафрейм `df_comm`. 

In [34]:
# по аналогии с циклом ранее, только ничего не увеличиваем
# подставляем на место post_id значение i

comments_all = []

for i in ids:
    params_comm = {"owner_id" : -38936316,
              "post_id" : i,
              "v" : v,
              "access_token" : token,
              "count" : 100, 
              "thread_items_count" : 10}
    req = requests.get(main_comm, params = params_comm)
    comm_json = req.json() 
    comm_list = comm_json["response"]["items"]
    comments_all.extend(comm_list)
    time.sleep(1)

In [35]:
df_comm = pd.DataFrame(comments_all)

In [36]:
# смотрим на названия столбцов
# здесь id – id поста

with_comm10.head(2)

,id,date,text,nlikes,nreposts,ncomments
1,4073,1686319330,Выберите правдивые утверждения о нашем тренере...,4,0,1
3,4071,1686220008,,25,0,2


In [37]:
# смотрим на названия столбцов
# здесь id – id комментария, а post_id - id поста

df_comm.head(2)

,id,from_id,date,text,post_id,owner_id,parents_stack,thread,attachments
0,4074,-38936316,1686319497,"Правдивые утверждения: 1, 3, 6, 8",4073,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN
1,4075,2847724,1686320865,🤣,4071,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN


### Задача 7

Объедините датафрейм с собранными комментариями `df_comm` и датафрейм `with_comm10` через функцию `merge()` из `pandas`.

In [38]:
# переименовываем столбцы в with_comm10, чтобы 
# столбец с id поста назывался везде одинаково
# inplace = True: сохраняем изменения

with_comm10.rename(columns = {"id" : "post_id"}, inplace = True)
with_comm10.head(2)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,post_id,date,text,nlikes,nreposts,ncomments
1,4073,1686319330,Выберите правдивые утверждения о нашем тренере...,4,0,1
3,4071,1686220008,,25,0,2


In [39]:
# объединяем with_comm10 и df_comm по столбцу post_id
# left: к левому датафрейму (with_comm10) подтягиваем данные из правого (df_comm)

final = with_comm10.merge(df_comm, on = "post_id", how = "left")

In [40]:
final

,post_id,date_x,text_x,nlikes,nreposts,ncomments,id,from_id,date_y,text_y,owner_id,parents_stack,thread,attachments
0,4073,1686319330,Выберите правдивые утверждения о нашем тренере...,4,0,1,4074,-38936316,1686319497,"Правдивые утверждения: 1, 3, 6, 8",-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN
1,4071,1686220008,,25,0,2,4075,2847724,1686320865,🤣,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN
2,4071,1686220008,,25,0,2,4077,260091507,1686333308,Класс! 😁,-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN
3,4058,1685624479,Как возраст влияет на скалолазание.\n\n🔹Статис...,35,11,4,4059,271412079,1685722035,По три процента в год в течении 30 лет это 90 ...,-38936316,[],"{'count': 1, 'items': [{'id': 4070, 'from_id':...",NaN
4,4058,1685624479,Как возраст влияет на скалолазание.\n\n🔹Статис...,35,11,4,4060,664352606,1685722985,"Правильно, нас уже давно списали",-38936316,[],"{'count': 1, 'items': [{'id': 4069, 'from_id':...",NaN
5,4052,1685519032,🔥 Мы подготовили новые трассы! \n\n1 x 5c\n4 ...,13,1,4,4053,260091507,1685526541,"Уже опробовал, хорошие, интересные! Фиолетовая...",-38936316,[],"{'count': 1, 'items': [{'id': 4057, 'from_id':...",NaN
6,4052,1685519032,🔥 Мы подготовили новые трассы! \n\n1 x 5c\n4 ...,13,1,4,4054,19182024,1685534085,Не успеваю всё попробовать 🧗‍♂️ \nКлассные тра...,-38936316,[],"{'count': 1, 'items': [{'id': 4056, 'from_id':...",NaN
7,4050,1685370675,🔥🔥🔥 Мы решили оставить прыжки с джамп контеста...,22,2,1,4051,260091507,1685373277,"Считай, Jump contest afterparty challenge! 🙂",-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN
8,4045,1685267824,🏆 Результаты джамп контекста: \n\n✨ Женщины Li...,21,1,4,4046,260091507,1685287705,"Спасибо! Мне понравилось, классно попрыгали - ...",-38936316,[],"{'count': 0, 'items': [], 'can_post': True, 's...",NaN
9,4045,1685267824,🏆 Результаты джамп контекста: \n\n✨ Женщины Li...,21,1,4,4047,313967562,1685300174,"А вы эту стенку с прыжками сразу скрутите, или...",-38936316,[],"{'count': 2, 'items': [{'id': 4048, 'from_id':...",NaN


## Часть 3:  собираем информацию о пользователях

Для примера выберем пользователя с id 20473269 (пока не из датафрейма, это всего лишь я):

In [41]:
j = 20473269

Сформируем запрос:

In [42]:
# метод users.get
# все знакомые параметры
# в fields перечисляем дополнительню информацию,
# которую хотим получить
# перечисляем через запятую без пробелов

main_user = "https://api.vk.com/method/users.get"
params_user = {"access_token" : token,
               "v" : v,
               "user_id" : j,
               "fields" : "bdate,city,home_town,universities"
}
req2 = requests.get(main_user, params = params_user)
req2.json()

{'response': [{'id': 20473269,
   'bdate': '25.3.1994',
   'city': {'id': 1, 'title': 'Москва'},
   'home_town': 'Москва',
   'universities': [{'city': 1,
     'country': 1,
     'education_status': 'Студентка (бакалавр)',
     'education_status_id': 3,
     'graduation': 2016,
     'id': 128,
     'name': 'НИУ ВШЭ (ГУ-ВШЭ)'}],
   'first_name': 'Алла',
   'last_name': 'Тамбовцева',
   'can_access_closed': True,
   'is_closed': False}]}

### Задача 8

Выберите какого-нибудь пользователя, оставившего комментарий, из датафрейма и запросите по нему интересующую информацию.

In [ ]:
### YOUR CODE HERE ###